# CellType

lamindb provides access to the following public protein ontologies through [lnschema-bionty](https://github.com/laminlabs/lnschema-bionty):

1. [Cell Ontology](https://obophenotype.github.io/cell-ontology)

Here we show how to access and search CellType ontologies to standardize new data.

## Setup

In [ ]:
!lamin init --storage ./test-celltype --schema bionty

In [ ]:
import lnschema_bionty as lb
import pandas as pd

## PublicOntology objects

Let us create a public ontology accessor with {meth}`~docs:lnschema_bionty.dev.BioRegistry.public`, which chooses a default public ontology source from {class}`~docs:lnschema_bionty.PublicSource`.
It's a [PublicOntology](https://lamin.ai/docs/lnschema_bionty.dev.publicontology) object, which you can think about as a public registry:

In [ ]:
celltypes = lb.CellType.public(organism="all")
celltypes

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = celltypes.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `public.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = celltypes.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.cd8_positive_alpha_beta_t_cell

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["CD8-positive, alpha-beta T cell"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = celltypes.lookup(celltypes.ontology_id)

In [ ]:
lookup.cl_0000625

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
celltypes.search("CD8 positive T cell").head(3)

By default, search also covers synonyms:

In [ ]:
celltypes.search("Tc1 T lymphocyte").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
celltypes.search("Tc1 T lymphocyte", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
celltypes.search(
    "CD8 positive alpha beta T cell",
    field=celltypes.definition,
).head()

## Standardize CellType identifiers

Let us generate a `DataFrame` that stores a number of CellType identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
    index=[
        "Boettcher cell"
        "bone marrow cell"
        "interstitial cell of ovary"
        "pancreatic ductal cell"
        "This celltype does not exist",
    ]
)
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = celltypes.validate(df_orig.index, celltypes.name)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.PublicSource.filter(entity="CellType").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
public_source = lb.PublicSource.filter(
    source="cl", version="2023-04-20", organism="all"
).one()
celltypes= lb.CellType.public(public_source=public_source)
celltypes

The currently used ontologies can be displayed using:

In [ ]:
lb.PublicSource.filter(currently_used=True).df()

In [ ]:
!lamin delete --force test-celltype
!rm -r test-celltype